In [125]:
import pandas as pd
import ast
import datetime

# Load books info from csv files

After checking the dfs, we can find that in "bk_2019","bk_2020" and "bk_2021",the format of the cells in column "genre" looks like list but not actual list. So first of all, we need to convert them to list and then to strings.

In [ ]:
lists = {}
lists['genre']=bk_2019['genre']
df = pd.DataFrame(lists)
# convert the string representation of lists into actual lists
df['genre'] = df['genre'].apply(ast.literal_eval)
# join the lists into clean string
df['genre'] = df['genre'].apply(lambda x: ', '.join(x))
df
# print(df['genre'].apply(type))

In [81]:
bk_2019 = pd.read_csv("../data/raw_data/books_2019_df.csv")
# convert the string representation of lists into actual lists
bk_2019['genre'] = bk_2019['genre'].apply(ast.literal_eval)
# join the lists into clean string
bk_2019['genre'] = bk_2019['genre'].apply(lambda x: ', '.join(x))
bk_2019.head(3)
bk_2019.shape

(400, 12)

In [83]:
bk_2020 = pd.read_csv("../data/raw_data/books_2020_df.csv")

bk_2020['genre'] = bk_2020['genre'].apply(ast.literal_eval)
bk_2020['genre'] = bk_2020['genre'].apply(lambda x: ', '.join(x))
bk_2020.shape

(399, 12)

In [85]:
bk_2021 = pd.read_csv("../data/raw_data/books_2021_df.csv")
bk_2021['genre'] = bk_2021['genre'].apply(ast.literal_eval)
bk_2021['genre'] = bk_2021['genre'].apply(lambda x: ', '.join(x))
bk_2021.shape

(340, 12)

In [86]:
bk_2022 = pd.read_csv("../data/raw_data/books_2022_df.csv")
bk_2022.shape

(340, 12)

In [87]:
bk_2023 = pd.read_csv("../data/raw_data/books_2023_df.csv")
bk_2023.shape

(299, 12)

# Add column ["year"] for each df

In [88]:
bk_2019['year'] = 2019
bk_2020['year'] = 2020
bk_2021['year'] = 2021
bk_2022['year'] = 2022
bk_2023['year'] = 2023

# Create an ensemble df

In [135]:
df = pd.concat([bk_2019,bk_2020,bk_2021,bk_2022,bk_2023], ignore_index=True)
df.shape

(1778, 13)

In [136]:
df.columns

Index(['title', 'ISBN', 'author', 'genre', 'rating', 'number_of_ratings',
       'number_of_reviews', 'pages', 'format', 'publish info', 'description',
       'Votes', 'year'],
      dtype='object')

In [137]:
df.isna().sum()

title                0
ISBN                 6
author               0
genre                0
rating               0
number_of_ratings    0
number_of_reviews    0
pages                8
format               0
publish info         0
description          0
Votes                0
year                 0
dtype: int64

In [138]:
df.duplicated().sum()

0

In [139]:
df.nunique()

title                1687
ISBN                   15
author               1307
genre                1605
rating                241
number_of_ratings    1685
number_of_reviews    1616
pages                 546
format                  8
publish info          411
description          1690
Votes                1691
year                    5
dtype: int64

# Cleaning Dataframe

- drop column "ISBN"
- We can see that most of the cells filled in the "ISBN not found", so we can drop it.

In [140]:
df = df.drop(columns=['ISBN'])

In [141]:
df.head(3)

,title,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,publish info,description,Votes,year
0,The Testaments,Margaret Atwood,"Fiction, Dystopia, Science Fiction, Feminism, ...",4.19,352440,31334,422.0,Hardcover,"First published September 10, 2019",When the van door slammed on Offred's future a...,98291,2019
1,Normal People,Sally Rooney,"Fiction, Romance, Contemporary, Literary Ficti...",3.81,1388906,135364,273.0,Hardcover,"First published August 28, 2018",At school Connell and Marianne pretend not to ...,40081,2019
2,Where the Forest Meets the Stars,Glendy Vanderah,"Fiction, Contemporary, Mystery, Romance, Fanta...",4.14,192265,16504,332.0,Hardcover,"First published March 1, 2019","In this gorgeously stunning debut, a mysteriou...",29342,2019


- deal with column "genre"
- extract the value of column and create a new df about genre of books

In [142]:
# split the 'genre' strings into lists of individual genres
genres_list = df['genre'].str.split(', ').explode().unique()

# create a new DataFrame with unique genres as a category
genres_df = pd.DataFrame(genres_list, columns=['genre'])

# looks like there is an empty string, convert it to null value
genres_df= genres_df.replace('', pd.NA).dropna(subset=['genre'])

genres_df['id'] = range(1, len(genres_df) + 1)

# change the order of columns
genres_df = genres_df[['id','genre']]

# export as csv file
genres_df.to_csv('../data/database/book_genres.csv', index=False)

- deal with column "publish info"
- keep only the publish date and convert it to datetime type

In [143]:
df['publish_date'] = df['publish info'].str.extract(r'(\w+ \d{1,2}, \d{4})')
df['publish_date'] = pd.to_datetime(df['publish_date'])

# drop "publish info"
df = df.drop(columns=['publish info'])

- drop the rows with "not found"

In [145]:
df['rating'].value_counts

<bound method IndexOpsMixin.value_counts of 0       4.19
1       3.81
2       4.14
3       3.94
4       3.86
        ... 
1773    4.16
1774    3.64
1775    3.79
1776     3.7
1777    3.74
Name: rating, Length: 1778, dtype: object>

- format the names of columns

In [134]:
# change the order of columns

,title,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,description,Votes,year,publish_date
0,The Testaments,Margaret Atwood,"Fiction, Dystopia, Science Fiction, Feminism, ...",4.19,352440,31334,422.0,Hardcover,When the van door slammed on Offred's future a...,98291,2019,2019-09-10
1,Normal People,Sally Rooney,"Fiction, Romance, Contemporary, Literary Ficti...",3.81,1388906,135364,273.0,Hardcover,At school Connell and Marianne pretend not to ...,40081,2019,2018-08-28
2,Where the Forest Meets the Stars,Glendy Vanderah,"Fiction, Contemporary, Mystery, Romance, Fanta...",4.14,192265,16504,332.0,Hardcover,"In this gorgeously stunning debut, a mysteriou...",29342,2019,2019-03-01
3,"Ask Again, Yes",Mary Beth Keane,"Fiction, Contemporary, Audiobook, Literary Fic...",3.94,192485,16702,388.0,Hardcover,"Francis Gleeson and Brian Stanhope, two rookie...",26653,2019,2019-05-28
4,Queenie,Candice Carty-Williams,"Fiction, Contemporary, Audiobook, Adult, Menta...",3.86,142313,13672,330.0,Hardcover,Bridget Jones’s Diary meets Americanah in this...,21524,2019,2019-03-19


In [146]:
# export as csv file
df.to_csv('../data/database/book_info.csv', index=False)